In [1]:
import pandas as pd
import numpy as np
import os
import re # Import regular expression module

# --- Configuration ---
input_csv_filename = 'P001.csv'
output_base_filename = os.path.splitext(input_csv_filename)[0] # Get 'P001'
output_csv_filename = f'{output_base_filename}_summary.csv'
output_xlsx_filename = f'{output_base_filename}_summary.xlsx'

# --- Volunteer ID ---
volunteer_id = output_base_filename

print(f"Processing file: {input_csv_filename} for Volunteer ID: {volunteer_id}")

# --- Data Processing ---
try:
    # 1. Read CSV
    # Use low_memory=False if the file is very large and causes dtype warnings
    df = pd.read_csv(input_csv_filename, low_memory=False)
    print(f"Read {len(df)} rows initially.")

    # 2. Remove rows with empty/NaN annotation
    df.dropna(subset=['annotation'], inplace=True)
    print(f"Kept {len(df)} rows after removing empty annotations.")

    # Check if any rows remain
    if df.empty:
        print("No valid annotations found. Cannot proceed.")
        # Create an empty/zeroed summary if needed, or exit
        summary_data = {
            '志愿者 ID': [volunteer_id],
            '记录总时长 (小时)': [0.0],
            '睡眠总时长 (小时)': [0.0],
            '高等强度运动总时长 (小时)': [0.0],
            '中等强度运动总时长 (小时)': [0.0],
            '低等强度运动总时长 (小时)': [0.0],
            '静态活动总时长 (小时)': [0.0]
        }
        summary_df = pd.DataFrame(summary_data)

    else:
        # 3. Extract MET value
        def extract_met(annotation):
            try:
                # Split by semicolon and take the last part
                last_part = annotation.split(';')[-1].strip()
                # Use regex to find a number (integer or float) potentially after "MET"
                match = re.search(r'(\d+(\.\d+)?)', last_part)
                if match:
                    return float(match.group(1))
                else:
                    # Handle cases where the format might differ slightly but contains a number
                    parts = last_part.split()
                    if parts and parts[-1].replace('.', '', 1).isdigit():
                         return float(parts[-1])
                    return np.nan # Return NaN if no number found
            except Exception:
                return np.nan # Return NaN in case of any errors

        df['MET'] = df['annotation'].apply(extract_met)

        # Remove rows where MET extraction failed
        initial_rows_with_met = len(df)
        df.dropna(subset=['MET'], inplace=True)
        if len(df) < initial_rows_with_met:
             print(f"Removed {initial_rows_with_met - len(df)} rows where MET could not be extracted.")
        print(f"Kept {len(df)} rows with valid MET values.")


        # 4. Classify activity based on MET value
        conditions = [
            df['MET'] < 1.0,                                  # Sleep
            (df['MET'] >= 1.0) & (df['MET'] < 1.6),           # Sedentary (Static)
            (df['MET'] >= 1.6) & (df['MET'] < 3.0),           # Low Intensity
            (df['MET'] >= 3.0) & (df['MET'] < 6.0),           # Medium Intensity
            df['MET'] >= 6.0                                  # High Intensity
        ]
        # Ensure the order matches the conditions
        choices = [
            'Sleep',
            'Sedentary',
            'Low Intensity',
            'Medium Intensity',
            'High Intensity'
        ]
        df['Activity Level'] = np.select(conditions, choices, default='Unknown')

        # Filter out any 'Unknown' categories if necessary (shouldn't happen with np.select)
        df = df[df['Activity Level'] != 'Unknown']

        # 5. Count rows per category
        activity_counts = df['Activity Level'].value_counts()

        # 6. Calculate duration in hours (row count * 0.01 seconds/row / 3600 seconds/hour)
        conversion_factor = 0.01 / 3600
        duration_hours = (activity_counts * conversion_factor).round(4) # Round to 4 decimal places

        # 7. Get durations for each specific category (use .get() for safety if a category has 0 counts)
        sleep_hours = duration_hours.get('Sleep', 0.0)
        high_intensity_hours = duration_hours.get('High Intensity', 0.0)
        medium_intensity_hours = duration_hours.get('Medium Intensity', 0.0)
        low_intensity_hours = duration_hours.get('Low Intensity', 0.0)
        sedentary_hours = duration_hours.get('Sedentary', 0.0) # Corrected based on MET 1.0-1.6

        # 8. Calculate total recorded duration
        total_hours = round(sleep_hours + high_intensity_hours + medium_intensity_hours + low_intensity_hours + sedentary_hours, 4)

        # 9. Create the summary DataFrame in the desired format
        summary_data = {
            '志愿者 ID': [volunteer_id],
            '记录总时长 (小时)': [total_hours],
            '睡眠总时长 (小时)': [sleep_hours],
            '高等强度运动总时长 (小时)': [high_intensity_hours],
            '中等强度运动总时长 (小时)': [medium_intensity_hours],
            '低等强度运动总时长 (小时)': [low_intensity_hours],
            '静态活动总时长 (小时)': [sedentary_hours] # Corrected column name for Sedentary
        }
        summary_df = pd.DataFrame(summary_data)

        # Reorder columns to match the example image exactly
        summary_df = summary_df[[
            '志愿者 ID',
            '记录总时长 (小时)',
            '睡眠总时长 (小时)',
            '高等强度运动总时长 (小时)',
            '中等强度运动总时长 (小时)',
            '低等强度运动总时长 (小时)',
            '静态活动总时长 (小时)'
        ]]

    # --- Output ---
    # 10. Save to CSV
    # Use encoding='utf-8-sig' for better compatibility with Excel opening Chinese characters
    summary_df.to_csv(output_csv_filename, index=False, encoding='utf-8-sig')
    print(f"Summary saved to CSV: {output_csv_filename}")

    # 11. Save to Excel
    summary_df.to_excel(output_xlsx_filename, index=False, engine='openpyxl')
    print(f"Summary saved to Excel: {output_xlsx_filename}")

    print("\nFinal Summary:")
    print(summary_df.to_string(index=False))

except FileNotFoundError:
    print(f"Error: Input file '{input_csv_filename}' not found.")
except Exception as e:
    print(f"An error occurred: {e}")

Processing file: P001.csv for Volunteer ID: P001
Read 10680001 rows initially.
Kept 8897719 rows after removing empty annotations.
Kept 8897719 rows with valid MET values.
Summary saved to CSV: P001_summary.csv
Summary saved to Excel: P001_summary.xlsx

Final Summary:
志愿者 ID  记录总时长 (小时)  睡眠总时长 (小时)  高等强度运动总时长 (小时)  中等强度运动总时长 (小时)  低等强度运动总时长 (小时)  静态活动总时长 (小时)
  P001     24.7158     10.5833             0.0          3.7303          3.0009        7.4013


In [4]:
import pandas as pd
import numpy as np
import os
import re # Import regular expression module

# --- Configuration ---
input_csv_filename = 'P002.csv'
output_base_filename = os.path.splitext(input_csv_filename)[0] # Get 'P001'
output_csv_filename = f'{output_base_filename}_summary.csv'
output_xlsx_filename = f'{output_base_filename}_summary.xlsx'

# --- Volunteer ID ---
volunteer_id = output_base_filename

print(f"Processing file: {input_csv_filename} for Volunteer ID: {volunteer_id}")

# --- Data Processing ---
try:
    # 1. Read CSV
    # Use low_memory=False if the file is very large and causes dtype warnings
    df = pd.read_csv(input_csv_filename, low_memory=False)
    print(f"Read {len(df)} rows initially.")

    # 2. Remove rows with empty/NaN annotation
    df.dropna(subset=['annotation'], inplace=True)
    print(f"Kept {len(df)} rows after removing empty annotations.")

    # Check if any rows remain
    if df.empty:
        print("No valid annotations found. Cannot proceed.")
        # Create an empty/zeroed summary if needed, or exit
        summary_data = {
            '志愿者 ID': [volunteer_id],
            '记录总时长 (小时)': [0.0],
            '睡眠总时长 (小时)': [0.0],
            '高等强度运动总时长 (小时)': [0.0],
            '中等强度运动总时长 (小时)': [0.0],
            '低等强度运动总时长 (小时)': [0.0],
            '静态活动总时长 (小时)': [0.0]
        }
        summary_df = pd.DataFrame(summary_data)

    else:
        # 3. Extract MET value
        def extract_met(annotation):
            try:
                # Split by semicolon and take the last part
                last_part = annotation.split(';')[-1].strip()
                # Use regex to find a number (integer or float) potentially after "MET"
                match = re.search(r'(\d+(\.\d+)?)', last_part)
                if match:
                    return float(match.group(1))
                else:
                    # Handle cases where the format might differ slightly but contains a number
                    parts = last_part.split()
                    if parts and parts[-1].replace('.', '', 1).isdigit():
                         return float(parts[-1])
                    return np.nan # Return NaN if no number found
            except Exception:
                return np.nan # Return NaN in case of any errors

        df['MET'] = df['annotation'].apply(extract_met)

        # Remove rows where MET extraction failed
        initial_rows_with_met = len(df)
        df.dropna(subset=['MET'], inplace=True)
        if len(df) < initial_rows_with_met:
             print(f"Removed {initial_rows_with_met - len(df)} rows where MET could not be extracted.")
        print(f"Kept {len(df)} rows with valid MET values.")


        # 4. Classify activity based on MET value
        conditions = [
            df['MET'] < 1.0,                                  # Sleep
            (df['MET'] >= 1.0) & (df['MET'] < 1.6),           # Sedentary (Static)
            (df['MET'] >= 1.6) & (df['MET'] < 3.0),           # Low Intensity
            (df['MET'] >= 3.0) & (df['MET'] < 6.0),           # Medium Intensity
            df['MET'] >= 6.0                                  # High Intensity
        ]
        # Ensure the order matches the conditions
        choices = [
            'Sleep',
            'Sedentary',
            'Low Intensity',
            'Medium Intensity',
            'High Intensity'
        ]
        df['Activity Level'] = np.select(conditions, choices, default='Unknown')

        # Filter out any 'Unknown' categories if necessary (shouldn't happen with np.select)
        df = df[df['Activity Level'] != 'Unknown']

        # 5. Count rows per category
        activity_counts = df['Activity Level'].value_counts()

        # 6. Calculate duration in hours (row count * 0.01 seconds/row / 3600 seconds/hour)
        conversion_factor = 0.01 / 3600
        duration_hours = (activity_counts * conversion_factor).round(4) # Round to 4 decimal places

        # 7. Get durations for each specific category (use .get() for safety if a category has 0 counts)
        sleep_hours = duration_hours.get('Sleep', 0.0)
        high_intensity_hours = duration_hours.get('High Intensity', 0.0)
        medium_intensity_hours = duration_hours.get('Medium Intensity', 0.0)
        low_intensity_hours = duration_hours.get('Low Intensity', 0.0)
        sedentary_hours = duration_hours.get('Sedentary', 0.0) # Corrected based on MET 1.0-1.6

        # 8. Calculate total recorded duration
        total_hours = round(sleep_hours + high_intensity_hours + medium_intensity_hours + low_intensity_hours + sedentary_hours, 4)

        # 9. Create the summary DataFrame in the desired format
        summary_data = {
            '志愿者 ID': [volunteer_id],
            '记录总时长 (小时)': [total_hours],
            '睡眠总时长 (小时)': [sleep_hours],
            '高等强度运动总时长 (小时)': [high_intensity_hours],
            '中等强度运动总时长 (小时)': [medium_intensity_hours],
            '低等强度运动总时长 (小时)': [low_intensity_hours],
            '静态活动总时长 (小时)': [sedentary_hours] # Corrected column name for Sedentary
        }
        summary_df = pd.DataFrame(summary_data)

        # Reorder columns to match the example image exactly
        summary_df = summary_df[[
            '志愿者 ID',
            '记录总时长 (小时)',
            '睡眠总时长 (小时)',
            '高等强度运动总时长 (小时)',
            '中等强度运动总时长 (小时)',
            '低等强度运动总时长 (小时)',
            '静态活动总时长 (小时)'
        ]]

    # --- Output ---
    # 10. Save to CSV
    # Use encoding='utf-8-sig' for better compatibility with Excel opening Chinese characters
    summary_df.to_csv(output_csv_filename, index=False, encoding='utf-8-sig')
    print(f"Summary saved to CSV: {output_csv_filename}")

    # 11. Save to Excel
    summary_df.to_excel(output_xlsx_filename, index=False, engine='openpyxl')
    print(f"Summary saved to Excel: {output_xlsx_filename}")

    print("\nFinal Summary:")
    print(summary_df.to_string(index=False))

except FileNotFoundError:
    print(f"Error: Input file '{input_csv_filename}' not found.")
except Exception as e:
    print(f"An error occurred: {e}")

Processing file: P002.csv for Volunteer ID: P002
Read 9390001 rows initially.
Kept 5810631 rows after removing empty annotations.
Kept 5810631 rows with valid MET values.
Summary saved to CSV: P002_summary.csv
Summary saved to Excel: P002_summary.xlsx

Final Summary:
志愿者 ID  记录总时长 (小时)  睡眠总时长 (小时)  高等强度运动总时长 (小时)  中等强度运动总时长 (小时)  低等强度运动总时长 (小时)  静态活动总时长 (小时)
  P002     16.1406        6.25          0.3567          1.1702          1.8108        6.5529


In [4]:
import pandas as pd
import numpy as np
import os
import glob # Import the glob module to find files
import re

# --- Configuration ---
output_csv_filename = 'result1.csv'
output_xlsx_filename = 'result1.xlsx'
file_pattern = 'P*.csv' # Pattern to find input CSV files

print(f"Starting processing for files matching: {file_pattern}")

# --- Find Input Files ---
csv_files = glob.glob(file_pattern)
if not csv_files:
    print(f"No files found matching '{file_pattern}' in the current directory.")
    exit() # Exit if no files are found

print(f"Found {len(csv_files)} files to process: {', '.join(csv_files)}")

# --- Data Processing Loop ---
all_summaries = [] # List to store summary DataFrames from each file

for input_csv_filename in csv_files:
    # Extract Volunteer ID (robustly handles potential path separators)
    base_name = os.path.basename(input_csv_filename)
    volunteer_id = os.path.splitext(base_name)[0] # Get 'P001', 'P002' etc.

    print(f"\n--- Processing file: {input_csv_filename} for Volunteer ID: {volunteer_id} ---")

    try:
        # 1. Read CSV
        df = pd.read_csv(input_csv_filename, low_memory=False)
        print(f"Read {len(df)} rows initially.")

        # 2. Remove rows with empty/NaN annotation
        df.dropna(subset=['annotation'], inplace=True)
        print(f"Kept {len(df)} rows after removing empty annotations.")

        # Check if any rows remain after filtering annotations
        if df.empty:
            print(f"No valid annotations found in {input_csv_filename}. Adding zero row.")
            summary_data = {
                '志愿者 ID': [volunteer_id], '记录总时长 (小时)': [0.0], '睡眠总时长 (小时)': [0.0],
                '高等强度运动总时长 (小时)': [0.0], '中等强度运动总时长 (小时)': [0.0],
                '低等强度运动总时长 (小时)': [0.0], '静态活动总时长 (小时)': [0.0]
            }
            summary_df = pd.DataFrame(summary_data)
            all_summaries.append(summary_df)
            continue # Move to the next file

        # 3. Extract MET value
        def extract_met(annotation):
            try:
                last_part = str(annotation).split(';')[-1].strip() # Ensure annotation is string
                match = re.search(r'(\d+(\.\d+)?)', last_part)
                if match:
                    return float(match.group(1))
                else:
                    parts = last_part.split()
                    # Check if the last part is a number (float or int)
                    if parts and parts[-1].replace('.', '', 1).isdigit():
                         return float(parts[-1])
                    return np.nan
            except Exception:
                return np.nan # Return NaN in case of errors like non-string input

        df['MET'] = df['annotation'].apply(extract_met)

        # Remove rows where MET extraction failed
        initial_rows_with_met = len(df)
        df.dropna(subset=['MET'], inplace=True)
        rows_removed_met = initial_rows_with_met - len(df)
        if rows_removed_met > 0:
             print(f"Removed {rows_removed_met} rows where MET could not be extracted.")
        print(f"Kept {len(df)} rows with valid MET values.")

        # Check if any rows remain after MET extraction
        if df.empty:
            print(f"No rows with valid MET values found in {input_csv_filename} after extraction. Adding zero row.")
            summary_data = {
                '志愿者 ID': [volunteer_id], '记录总时长 (小时)': [0.0], '睡眠总时长 (小时)': [0.0],
                '高等强度运动总时长 (小时)': [0.0], '中等强度运动总时长 (小时)': [0.0],
                '低等强度运动总时长 (小时)': [0.0], '静态活动总时长 (小时)': [0.0]
            }
            summary_df = pd.DataFrame(summary_data)
            all_summaries.append(summary_df)
            continue # Move to the next file

        # 4. Classify activity based on MET value
        conditions = [
            df['MET'] < 1.0,
            (df['MET'] >= 1.0) & (df['MET'] < 1.6),
            (df['MET'] >= 1.6) & (df['MET'] < 3.0),
            (df['MET'] >= 3.0) & (df['MET'] < 6.0),
            df['MET'] >= 6.0
        ]
        choices = ['Sleep', 'Sedentary', 'Low Intensity', 'Medium Intensity', 'High Intensity']
        df['Activity Level'] = np.select(conditions, choices, default='Unknown')
        df = df[df['Activity Level'] != 'Unknown']

        # 5. Count rows per category
        activity_counts = df['Activity Level'].value_counts()

        # 6. Calculate duration in hours
        conversion_factor = 0.01 / 3600
        duration_hours = (activity_counts * conversion_factor).round(4)

        # 7. Get durations for each specific category
        sleep_hours = duration_hours.get('Sleep', 0.0)
        high_intensity_hours = duration_hours.get('High Intensity', 0.0)
        medium_intensity_hours = duration_hours.get('Medium Intensity', 0.0)
        low_intensity_hours = duration_hours.get('Low Intensity', 0.0)
        sedentary_hours = duration_hours.get('Sedentary', 0.0)

        # 8. Calculate total recorded duration
        total_hours = round(sleep_hours + high_intensity_hours + medium_intensity_hours + low_intensity_hours + sedentary_hours, 4)

        # 9. Create the single-row summary DataFrame for this file
        summary_data = {
            '志愿者 ID': [volunteer_id],
            '记录总时长 (小时)': [total_hours],
            '睡眠总时长 (小时)': [sleep_hours],
            '高等强度运动总时长 (小时)': [high_intensity_hours],
            '中等强度运动总时长 (小时)': [medium_intensity_hours],
            '低等强度运动总时长 (小时)': [low_intensity_hours],
            '静态活动总时长 (小时)': [sedentary_hours]
        }
        summary_df = pd.DataFrame(summary_data)

        # Reorder columns (optional, but good for consistency)
        summary_df = summary_df[[
            '志愿者 ID', '记录总时长 (小时)', '睡眠总时长 (小时)',
            '高等强度运动总时长 (小时)', '中等强度运动总时长 (小时)',
            '低等强度运动总时长 (小时)', '静态活动总时长 (小时)'
        ]]

        # 10. Append the summary for this file to the main list
        all_summaries.append(summary_df)
        print(f"Finished processing {input_csv_filename}. Summary added.")

    except FileNotFoundError:
        print(f"Error: Input file '{input_csv_filename}' not found or inaccessible.")
    except pd.errors.EmptyDataError:
         print(f"Warning: File '{input_csv_filename}' is empty. Adding zero row.")
         summary_data = { # Add zero row if file is empty
                '志愿者 ID': [volunteer_id], '记录总时长 (小时)': [0.0], '睡眠总时长 (小时)': [0.0],
                '高等强度运动总时长 (小时)': [0.0], '中等强度运动总时长 (小时)': [0.0],
                '低等强度运动总时长 (小时)': [0.0], '静态活动总时长 (小时)': [0.0]
         }
         summary_df = pd.DataFrame(summary_data)
         all_summaries.append(summary_df)
    except Exception as e:
        print(f"An error occurred processing file '{input_csv_filename}': {e}")
        print("Attempting to add zero row for this file.")
        try: # Try adding a zero row even if processing failed partially
            summary_data = {
                    '志愿者 ID': [volunteer_id], '记录总时长 (小时)': [0.0], '睡眠总时长 (小时)': [0.0],
                    '高等强度运动总时长 (小时)': [0.0], '中等强度运动总时长 (小时)': [0.0],
                    '低等强度运动总时长 (小时)': [0.0], '静态活动总时长 (小时)': [0.0]
            }
            summary_df = pd.DataFrame(summary_data)
            all_summaries.append(summary_df)
        except Exception as inner_e:
             print(f"Could not even add zero row for {volunteer_id} due to: {inner_e}")


# --- Final Aggregation and Output ---
if not all_summaries:
    print("\nNo files resulted in usable summaries. No output generated.")
else:
    # Concatenate all individual summary DataFrames into one
    final_summary_df = pd.concat(all_summaries, ignore_index=True)

    # --- Add Sorting Step Here ---
    print("\nSorting final results by '志愿者 ID'...")
    final_summary_df = final_summary_df.sort_values(by='志愿者 ID', ascending=True)
    # --- Sorting Done ---

    print("\n--- Final Combined and Sorted Summary ---")
    # Use to_string to display more rows/cols if needed in console
    # print(final_summary_df.to_string(index=False))
    # For potentially long tables, just printing head might be better
    print(final_summary_df.head().to_string(index=False))
    if len(final_summary_df) > 5:
        print("...")


    # Save the combined and sorted DataFrame to CSV and Excel
    try:
        final_summary_df.to_csv(output_csv_filename, index=False, encoding='utf-8-sig')
        print(f"\nCombined summary saved to CSV: {output_csv_filename}")

        final_summary_df.to_excel(output_xlsx_filename, index=False, engine='openpyxl')
        print(f"Combined summary saved to Excel: {output_xlsx_filename}")
    except Exception as e:
        print(f"\nError saving output files: {e}")

Starting processing for files matching: P*.csv
Found 4 files to process: P001_summary.csv, P002_summary.csv, P002.csv, P001.csv

--- Processing file: P001_summary.csv for Volunteer ID: P001_summary ---
Read 1 rows initially.
An error occurred processing file 'P001_summary.csv': ['annotation']
Attempting to add zero row for this file.

--- Processing file: P002_summary.csv for Volunteer ID: P002_summary ---
Read 1 rows initially.
An error occurred processing file 'P002_summary.csv': ['annotation']
Attempting to add zero row for this file.

--- Processing file: P002.csv for Volunteer ID: P002 ---
Read 9390001 rows initially.
Kept 5810631 rows after removing empty annotations.
Kept 5810631 rows with valid MET values.
Finished processing P002.csv. Summary added.

--- Processing file: P001.csv for Volunteer ID: P001 ---
Read 10680001 rows initially.
Kept 8897719 rows after removing empty annotations.
Kept 8897719 rows with valid MET values.
Finished processing P001.csv. Summary added.

Sort